# Preprocessing: Neighborhood characteristics.
First let's start with imports and loading the data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
characteristics = pd.read_csv('characteristics.csv')
characteristics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   cty                           3221 non-null   object 
 1   name                          3220 non-null   object 
 2   median_rent2016               3218 non-null   float64
 3   nonwhite_share2010            3220 non-null   float64
 4   mail_return_rate2010          3193 non-null   float64
 5   med_hhinc1990_real            3142 non-null   float64
 6   lfp2010                       3220 non-null   float64
 7   job_density_2013              3218 non-null   float64
 8   foreign_share2016             3141 non-null   float64
 9   popdensity2010                3220 non-null   float64
 10  traveltime15_2016             3219 non-null   float64
 11  poor_share2016                3219 non-null   float64
 12  id2                           3221 non-null   object 
 13  ann

In [ ]:
# Filter scope of data down to California and extract counties
characteristics[['county', 'state']] = characteristics['name'].str.split(', ', expand=True)
characteristics = characteristics[characteristics['state'] == 'CA']

# Select columns and rename them
characteristics = characteristics.loc[:, ['county', 'med_hhinc1990_real', 'ann_avg_job_growth_2004_2013', 'nonwhite_share2010', 'popdensity2010']]
characteristics.rename(columns={'med_hhinc1990_real': 'Median_Household_Income_1990', 'ann_avg_job_growth_2004_2013': 'Avg_Job_Growth_2004_2013', 'nonwhite_share2010': 'Fraction_Nonwhite_2010', 'popdensity2010': 'Population_Density_2010'}, inplace=True)

In [ ]:
characteristics.dropna(inplace=True)
characteristics.sort_values(by='county').head(5)

,county,Median_Household_Income_1990,Avg_Job_Growth_2004_2013,Fraction_Nonwhite_2010,Population_Density_2010
187,Alameda County,74280.0,0.0072,0.6593,2044.000
188,Alpine County,45222.0,-0.0037,0.2749,1.591
189,Amador County,55000.0,-0.0227,0.2039,64.060
190,Butte County,43488.0,0.0012,0.2481,134.400
191,Calaveras County,52410.0,-0.0022,0.1646,44.680


In [ ]:
characteristics.to_csv('characteristics_clean.csv', index=False)

# Preprocessing: Air Quality.
First let's start with imports and loading the data

In [ ]:
air_quality = pd.read_csv('concentrations.csv')
air_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296554 entries, 0 to 296553
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            296554 non-null  int64  
 1   date            296554 non-null  object 
 2   statefips       296554 non-null  int64  
 3   countyfips      296554 non-null  int64  
 4   pm25_max_pred   296554 non-null  float64
 5   pm25_med_pred   296554 non-null  float64
 6   pm25_mean_pred  296554 non-null  float64
 7   pm25_pop_pred   296554 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 18.1+ MB


In [ ]:
air_quality.head(5)

,year,date,statefips,countyfips,pm25_max_pred,pm25_med_pred,pm25_mean_pred,pm25_pop_pred
0,2001,01JAN2001,6,6001,56.708484,52.434557,52.751494,52.942986
1,2001,01JAN2001,6,6003,12.663324,12.663324,12.663324,12.663324
2,2001,01JAN2001,6,6005,25.662103,21.031934,21.176171,21.894553
3,2001,01JAN2001,6,6007,26.702092,25.527019,25.356636,25.364373
4,2001,01JAN2001,6,6009,24.662062,18.489823,19.373269,20.626233


In [ ]:
county_mapping = {
    6001: "Alameda County",
    6003: "Alpine County",
    6005: "Amador County",
    6007: "Butte County",
    6009: "Calaveras County",
    6011: "Colusa County",
    6013: "Contra Costa County",
    6015: "Del Norte County",
    6017: "El Dorado County",
    6019: "Fresno County",
    6021: "Glenn County",
    6023: "Humboldt County",
    6025: "Imperial County",
    6027: "Inyo County",
    6029: "Kern County",
    6031: "Kings County",
    6033: "Lake County",
    6035: "Lassen County",
    6037: "Los Angeles County",
    6039: "Madera County",
    6041: "Marin County",
    6043: "Mariposa County",
    6045: "Mendocino County",
    6047: "Merced County",
    6049: "Modoc County",
    6051: "Mono County",
    6053: "Monterey County",
    6055: "Napa County",
    6057: "Nevada County",
    6059: "Orange County",
    6061: "Placer County",
    6063: "Plumas County",
    6065: "Riverside County",
    6067: "Sacramento County",
    6069: "San Benito County",
    6071: "San Bernardino County",
    6073: "San Diego County",
    6075: "San Francisco County",
    6077: "San Joaquin County",
    6079: "San Luis Obispo County",
    6081: "San Mateo County",
    6083: "Santa Barbara County",
    6085: "Santa Clara County",
    6087: "Santa Cruz County",
    6089: "Shasta County",
    6091: "Sierra County",
    6093: "Siskiyou County",
    6095: "Solano County",
    6097: "Sonoma County",
    6099: "Stanislaus County",
    6101: "Sutter County",
    6103: "Tehama County",
    6105: "Trinity County",
    6107: "Tulare County",
    6109: "Tuolumne County",
    6111: "Ventura County",
    6113: "Yolo County",
    6115: "Yuba County"
}

In [ ]:
air_quality['county'] = air_quality['countyfips'].map(county_mapping)
air_quality = air_quality.groupby(['county', 'year'])['pm25_mean_pred'].mean().reset_index()
air_quality.rename(columns={'pm25_mean_pred': 'pm25_mean'}, inplace=True)

In [ ]:
air_quality.head(5)

,county,year,pm25_mean
0,Alameda County,2001,11.337368
1,Alameda County,2002,12.515194
2,Alameda County,2003,11.746714
3,Alameda County,2004,10.519771
4,Alameda County,2005,10.046940


In [ ]:
air_quality.to_csv('air_quality_short.csv', index=False)
